# Visual Feature Extraction

mp4 -> fps -> npy

In [ ]:
import time
import numpy as np
import cv2
import pandas as pd
from pathlib import Path
import warnings

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tqdm import tqdm # Pakai versi notebook biar keren di Jupyter

warnings.filterwarnings("ignore")

# 1. SETUP MODEL & PATH
print("Memuat model ResNet50...")
# Buat base model 
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
# Definisikan model dari base model yang sama
visual_model = Model(inputs=base_model.input, outputs=base_model.output)
IMG_SIZE = 224
print("✅ Model Visual (ResNet50) siap.")

# --- Path Setup ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
VIDEO_DIR = DATA_DIR / "video"
PROCESSED_DATA_DIR = DATA_DIR / "processed"
FEATURES_DIR = BASE_DIR / "features"

# Path folder input video
TRAIN_VIDEO_DIR = VIDEO_DIR / "train"
TEST_VIDEO_DIR = VIDEO_DIR / "test"

# Path folder output sementara untuk menyimpan progres
TEMP_VISUAL_TRAIN_DIR = FEATURES_DIR / "temp_visual_train"
TEMP_VISUAL_TEST_DIR = FEATURES_DIR / "temp_visual_test"

# Path file CSV
TRAIN_CSV_PATH = PROCESSED_DATA_DIR / "train_clean.csv"
TEST_CSV_PATH = PROCESSED_DATA_DIR / "test_clean.csv"

# Path file output final
TRAIN_VISUAL_FEATURES_PATH = FEATURES_DIR / "visual_train.npy"
TEST_VISUAL_FEATURES_PATH = FEATURES_DIR / "visual_test.npy"

# Membuat semua folder output yang dibutuhkan
FEATURES_DIR.mkdir(exist_ok=True)
TEMP_VISUAL_TRAIN_DIR.mkdir(exist_ok=True)
TEMP_VISUAL_TEST_DIR.mkdir(exist_ok=True)
print("Semua path dan folder output sudah siap.")

# 2. FUNGSI EKSTRAKSI 
def extract_visual_features(video_path, num_frames=30):
    cap = cv2.VideoCapture(str(video_path))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames < 1: return np.zeros(2048)
    
    indices = np.linspace(0, total_frames - 1, num=num_frames, dtype=int)
    frames = []
    for i in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frames.append(frame)
    cap.release()
    
    if not frames: return np.zeros(2048)
    
    # Proses dengan batching kecil untuk hemat memori
    features = visual_model.predict(preprocess_input(np.array(frames)), batch_size=8, verbose=0)
    return np.mean(features, axis=0)

# 3. TAHAP 1: EKSTRAK & SIMPAN PROGRES PER FILE
def process_and_save_individually(csv_path, video_dir, output_dir):
    df = pd.read_csv(csv_path)
    for video_id in tqdm(df['id'], desc=f"Ekstrak Visual dari {video_dir.name}"):
        video_path = video_dir / f"{video_id}.mp4"
        feature_path = output_dir / f"{video_id}.npy"
        
        # Cek apakah file ini sudah pernah diproses, jika iya, lewati
        if feature_path.exists():
            continue
            
        if not video_path.exists():
            features = np.zeros(2048) # Output ResNet50
        else:
            features = extract_visual_features(video_path)
        
        np.save(feature_path, features)

print("\n--- TAHAP 1: Memulai ekstraksi fitur visual satu per satu ---")
process_and_save_individually(TRAIN_CSV_PATH, TRAIN_VIDEO_DIR, TEMP_VISUAL_TRAIN_DIR)
process_and_save_individually(TEST_CSV_PATH, TEST_VIDEO_DIR, TEMP_VISUAL_TEST_DIR)
print("✅ Ekstraksi individual selesai.")

# 4. TAHAP 2: GABUNGKAN SEMUA HASIL
def combine_features(csv_path, temp_dir, output_path):
    df = pd.read_csv(csv_path)
    feature_list = []
    for video_id in tqdm(df['id'], desc=f"Menggabungkan fitur dari {temp_dir.name}"):
        feature = np.load(temp_dir / f"{video_id}.npy")
        feature_list.append(feature)
    
    final_features = np.array(feature_list)
    np.save(output_path, final_features)
    print(f"Fitur final disimpan di {output_path.name} dengan bentuk: {final_features.shape}")

print("\n--- TAHAP 2: Menggabungkan semua hasil fitur ---")
combine_features(TRAIN_CSV_PATH, TEMP_VISUAL_TRAIN_DIR, TRAIN_VISUAL_FEATURES_PATH)
combine_features(TEST_CSV_PATH, TEMP_VISUAL_TEST_DIR, TEST_VISUAL_FEATURES_PATH)

print("\n\nSELESAI! ✅ Fitur visual siap digunakan.")

Memuat model ResNet50...
✅ Model Visual (ResNet50) siap.
Semua path dan folder output sudah siap.

--- TAHAP 1: Memulai ekstraksi fitur visual satu per satu ---


Ekstrak Visual dari test: 100%|██████████| 200/200 [13:03<00:00,  3.92s/it]


✅ Ekstraksi individual selesai.

--- TAHAP 2: Menggabungkan semua hasil fitur ---


Menggabungkan fitur dari temp_visual_train: 100%|██████████| 802/802 [00:12<00:00, 62.83it/s]


Fitur final disimpan di visual_train.npy dengan bentuk: (802, 2048)


Menggabungkan fitur dari temp_visual_test: 100%|██████████| 200/200 [00:02<00:00, 69.36it/s]

Fitur final disimpan di visual_test.npy dengan bentuk: (200, 2048)


SELESAI! ✅ Fitur visual siap digunakan.


# Text Feature Extraction

mp4 -> txt

In [ ]:
import pandas as pd
from pathlib import Path
import warnings
import json
import whisper
from tqdm import tqdm

warnings.filterwarnings("ignore")

# 1. SETUP MODEL & PATH
print("Memuat model Whisper...")
try:
    model = whisper.load_model("base")
    print("✅ Model Teks (Whisper) siap.")
except Exception as e:
    print(f"❌ Gagal memuat model Whisper: {e}")

# --- Path Setup ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
VIDEO_DIR = DATA_DIR / "video" 
PROCESSED_DATA_DIR = DATA_DIR / "processed"
FEATURES_DIR = BASE_DIR / "features"

# Path folder input video
TRAIN_VIDEO_DIR = VIDEO_DIR / "train"
TEST_VIDEO_DIR = VIDEO_DIR / "test"

# Path folder output sementara untuk menyimpan transkrip per file
TEMP_TEXT_TRAIN_DIR = FEATURES_DIR / "temp_text_train"
TEMP_TEXT_TEST_DIR = FEATURES_DIR / "temp_text_test"

# Path file CSV
TRAIN_CSV_PATH = PROCESSED_DATA_DIR / "train_clean.csv"
TEST_CSV_PATH = PROCESSED_DATA_DIR / "test_clean.csv"

# Path file output final 
TRAIN_TEXT_FEATURES_PATH = FEATURES_DIR / "text_train.csv"
TEST_TEXT_FEATURES_PATH = FEATURES_DIR / "text_test.csv"

# Membuat semua folder output yang dibutuhkan
FEATURES_DIR.mkdir(exist_ok=True)
TEMP_TEXT_TRAIN_DIR.mkdir(exist_ok=True)
TEMP_TEXT_TEST_DIR.mkdir(exist_ok=True)
print("Semua path dan folder output sudah siap.")

# 2. TAHAP 1: TRANSKRIPSI & SIMPAN PROGRES PER FILE
def transcribe_and_save_individually(csv_path, video_dir, output_dir):
    df = pd.read_csv(csv_path)
    for video_id in tqdm(df['id'], desc=f"Transkripsi dari {video_dir.name}"):
        video_path = video_dir / f"{video_id}.mp4"
        # Kita simpan hasilnya sebagai file teks biasa (.txt)
        transcript_path = output_dir / f"{video_id}.txt"
        
        # Cek apakah file ini sudah pernah diproses, jika iya, lewati
        if transcript_path.exists():
            continue
            
        transcript_text = ""
        if video_path.exists():
            try:
                # Lakukan transkripsi
                result = model.transcribe(str(video_path), fp16=False)
                transcript_text = result['text']
            except Exception as e:
                print(f"Gagal transkripsi {video_path.name}: {e}")
                transcript_text = "" # Beri teks kosong jika gagal
        
        # Simpan hasil transkrip ke file .txt
        with open(transcript_path, 'w', encoding='utf-8') as f:
            f.write(transcript_text)

print("\n--- TAHAP 1: Memulai transkripsi audio ke teks satu per satu ---")
transcribe_and_save_individually(TRAIN_CSV_PATH, TRAIN_VIDEO_DIR, TEMP_TEXT_TRAIN_DIR)
transcribe_and_save_individually(TEST_CSV_PATH, TEST_VIDEO_DIR, TEMP_TEXT_TEST_DIR)
print("✅ Transkripsi individual selesai.")

# 3. TAHAP 2: GABUNGKAN SEMUA HASIL TRANSKRIP
def combine_transcripts(csv_path, temp_dir, output_path):
    df = pd.read_csv(csv_path)
    transcripts = []
    for video_id in tqdm(df['id'], desc=f"Menggabungkan transkrip dari {temp_dir.name}"):
        transcript_path = temp_dir / f"{video_id}.txt"
        try:
            with open(transcript_path, 'r', encoding='utf-8') as f:
                text = f.read()
        except FileNotFoundError:
            text = "" # Teks kosong jika file tidak ditemukan
        transcripts.append(text)
    
    # Buat DataFrame baru berisi id dan transkrip
    result_df = pd.DataFrame({
        'id': df['id'],
        'transcript': transcripts
    })
    
    # Simpan ke file CSV
    result_df.to_csv(output_path, index=False)
    print(f"Transkrip final disimpan di {output_path.name}")

print("\n--- TAHAP 2: Menggabungkan semua hasil transkrip ---")
combine_transcripts(TRAIN_CSV_PATH, TEMP_TEXT_TRAIN_DIR, TRAIN_TEXT_FEATURES_PATH)
combine_transcripts(TEST_CSV_PATH, TEMP_TEXT_TEST_DIR, TEST_TEXT_FEATURES_PATH)

print("\n\nSELESAI! ✅ Transkrip teks siap digunakan.")

Memuat model Whisper...
✅ Model Teks (Whisper) siap.
Semua path dan folder output sudah siap.

--- TAHAP 1: Memulai transkripsi audio ke teks satu per satu ---


Transkripsi dari test: 100%|██████████| 200/200 [45:49<00:00, 13.75s/it]


✅ Transkripsi individual selesai.

--- TAHAP 2: Menggabungkan semua hasil transkrip ---


Menggabungkan transkrip dari temp_text_train: 100%|██████████| 802/802 [00:11<00:00, 69.94it/s]


Transkrip final disimpan di text_train.csv


Menggabungkan transkrip dari temp_text_test: 100%|██████████| 200/200 [00:02<00:00, 75.64it/s]

Transkrip final disimpan di text_test.csv


SELESAI! ✅ Transkrip teks siap digunakan.


Ekstraksi Fitur Teks (IndoBERT Embedding)


txt -> npy

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import torch
from tqdm import tqdm
from transformers import BertTokenizer, BertModel

warnings.filterwarnings("ignore")

# 1. SETUP MODEL INDOBERT & PATH
print("Mengecek ketersediaan GPU untuk PyTorch...")
# Cek apakah GPU bisa digunakan oleh PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan device: {device}")

print("\nMemuat model IndoBERT...")
try:
    # Muat tokenizer dan model IndoBERT
    tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
    model = BertModel.from_pretrained('indobenchmark/indobert-base-p1')
    # Pindahkan model ke GPU jika tersedia
    model.to(device)
    print("✅ Model IndoBERT siap.")
except Exception as e:
    print(f"❌ Gagal memuat model IndoBERT: {e}")

# --- Path Setup ---
BASE_DIR = Path.cwd().parent
FEATURES_DIR = BASE_DIR / "features"

# Path file input (hasil dari Whisper)
TRAIN_TEXT_CSV_PATH = FEATURES_DIR / "text_train.csv"
TEST_TEXT_CSV_PATH = FEATURES_DIR / "text_test.csv"

# Path file output final (.npy)
TRAIN_TEXT_FEATURES_PATH = FEATURES_DIR / "text_train.npy"
TEST_TEXT_FEATURES_PATH = FEATURES_DIR / "text_test.npy"

print("Semua path sudah siap.")

# 2. FUNGSI UNTUK MENDAPATKAN EMBEDDING
def get_text_embedding(text):
    """
    Mengubah satu kalimat teks menjadi satu vektor fitur (embedding).
    """
    if pd.isna(text) or text.strip() == "":
        # IndoBERT base menghasilkan 768 fitur
        return np.zeros(768) 

    # Tokenisasi teks
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=128, padding=True)
    # Pindahkan input tensor ke GPU
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Dapatkan output dari model tanpa menghitung gradien (lebih hemat memori)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Ambil embedding dari token [CLS] (representasi keseluruhan kalimat)
    embedding = outputs.last_hidden_state[0, 0, :].cpu().numpy()
    return embedding

# 3. PROSES KESELURUHAN DATASET
def process_text_dataset(csv_path, output_path):
    df = pd.read_csv(csv_path)
    all_embeddings = []
    
    for text in tqdm(df['transcript'], desc=f"Membuat embedding dari {csv_path.name}"):
        embedding = get_text_embedding(text)
        all_embeddings.append(embedding)
    
    # Simpan hasil akhir ke file .npy
    final_embeddings = np.array(all_embeddings)
    np.save(output_path, final_embeddings)
    print(f"Fitur teks final disimpan di {output_path.name} dengan bentuk: {final_embeddings.shape}")


print("\n--- Memulai proses embedding untuk data latih ---")
process_text_dataset(TRAIN_TEXT_CSV_PATH, TRAIN_TEXT_FEATURES_PATH)

print("\n--- Memulai proses embedding untuk data tes ---")
process_text_dataset(TEST_TEXT_CSV_PATH, TEST_TEXT_FEATURES_PATH)

print("\n\nSELESAI! ✅ Semua fitur (audio, visual, teks) sudah lengkap dan siap.")

c:\Users\kn409\anaconda3\envs\satria_gpu\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\kn409\anaconda3\envs\satria_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Mengecek ketersediaan GPU untuk PyTorch...
Menggunakan device: cpu

Memuat model IndoBERT...
✅ Model IndoBERT siap.
Semua path sudah siap.

--- Memulai proses embedding untuk data latih ---


Membuat embedding dari text_train.csv: 100%|██████████| 802/802 [01:55<00:00,  6.92it/s]


Fitur teks final disimpan di text_train.npy dengan bentuk: (802, 768)

--- Memulai proses embedding untuk data tes ---


Membuat embedding dari text_test.csv: 100%|██████████| 200/200 [00:30<00:00,  6.58it/s]

Fitur teks final disimpan di text_test.npy dengan bentuk: (200, 768)


SELESAI! ✅ Semua fitur (audio, visual, teks) sudah lengkap dan siap.


# Audio Feature Extraction

mp4 -> wav

In [ ]:
import os
import subprocess
from pathlib import Path
from tqdm import tqdm

# --- Konfigurasi Path ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
VIDEO_DIR = DATA_DIR / "video"
WAV_DIR = DATA_DIR / "wav" # Folder output untuk file .wav

# List direktori yang akan diproses
video_subdirs = [VIDEO_DIR / "train", VIDEO_DIR / "test"]

print("Memulai proses konversi MP4 ke WAV...")

# Loop untuk folder train dan test
for video_subdir in video_subdirs:
    # Tentukan folder output untuk file .wav
    wav_output_dir = WAV_DIR / video_subdir.name
    # Buat folder output jika belum ada
    wav_output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\nMemproses folder: {video_subdir}")
    
    # Dapatkan daftar semua file .mp4 di dalam subdirektori
    video_files = list(video_subdir.glob("*.mp4"))
    
    for video_path in tqdm(video_files, desc=f"Mengonversi {video_subdir.name}"):
        video_id = video_path.stem # Mengambil nama file tanpa ekstensi (misal: "1")
        wav_path = wav_output_dir / f"{video_id}.wav"
        
        # Lewati file yang sudah pernah dikonversi
        if wav_path.exists():
            continue
            
        # Perintah FFMPEG untuk konversi
        # -i: input file
        # -vn: abaikan video (kita hanya butuh audio)
        # -acodec pcm_s16le: format audio standar untuk .wav
        # -ar 22050: sample rate 22.050 Hz
        # -ac 1: mono channel
        command = [
            "ffmpeg",
            "-i", str(video_path),
            "-vn",
            "-acodec", "pcm_s16le",
            "-ar", "22050",
            "-ac", "1",
            str(wav_path),
            "-hide_banner", # Sembunyikan info banner ffmpeg
            "-loglevel", "error" # Hanya tampilkan error jika ada
        ]
        
        try:
            # Menjalankan perintah ffmpeg
            subprocess.run(command, check=True)
        except subprocess.CalledProcessError as e:
            print(f"Gagal mengonversi {video_path.name}: {e}")

print("\n\nSemua video berhasil dikonversi ke format WAV! 🚀")

Memulai proses konversi MP4 ke WAV...

Memproses folder: c:\Caelan\BDC2025\data\video\train


Mengonversi train: 100%|██████████| 775/775 [05:25<00:00,  2.38it/s]



Memproses folder: c:\Caelan\BDC2025\data\video\test


Mengonversi test: 100%|██████████| 198/198 [01:21<00:00,  2.44it/s]



Semua video berhasil dikonversi ke format WAV! 🚀


diagnostik librosa

In [ ]:
import librosa
from pathlib import Path

file_tes = Path.cwd().parent / "data" / "wav" / "train" / "1.wav"

print("="*40)
print("MEMULAI TES DIAGNOSTIK LIBROSA")
print("="*40)
print(f"Mencoba memuat file: {file_tes}")
print(f"Apakah file ada? -> {file_tes.exists()}")

try:
    # Ini adalah perintah paling dasar untuk memuat audio.
    # Jika ini crash, masalah ada di librosa/environment.
    y, sr = librosa.load(file_tes)
    
    print("\n✅ BERHASIL! Audio berhasil dimuat tanpa crash.")
    print(f"   Sample rate: {sr}")
    print(f"   Jumlah sampel audio: {len(y)}")
    
except Exception as e:
    print(f"\n❌ GAGAL dengan error Python: {e}")

print("\nTes diagnostik selesai.")

MEMULAI TES DIAGNOSTIK LIBROSA
Mencoba memuat file: c:\Caelan\BDC2025\data\wav\train\1.wav
Apakah file ada? -> True

✅ BERHASIL! Audio berhasil dimuat tanpa crash.
   Sample rate: 22050
   Jumlah sampel audio: 2609214

Tes diagnostik selesai.


wav -> npy

In [ ]:
import pandas as pd
import numpy as np
import librosa
import os
from pathlib import Path
from tqdm import tqdm
import warnings

# Abaikan warning dari librosa yang tidak relevan
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# 1. SETUP PATH
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
WAV_DIR = DATA_DIR / "wav"
PROCESSED_DATA_DIR = DATA_DIR / "processed"
FEATURES_DIR = BASE_DIR / "features"

# Path folder input WAV
TRAIN_WAV_DIR = WAV_DIR / "train"
TEST_WAV_DIR = WAV_DIR / "test"

# Path folder output sementara untuk .npy per file
TEMP_TRAIN_DIR = FEATURES_DIR / "temp_train"
TEMP_TEST_DIR = FEATURES_DIR / "temp_test"

# Path file CSV
TRAIN_CSV_PATH = PROCESSED_DATA_DIR / "train_clean.csv"
TEST_CSV_PATH = PROCESSED_DATA_DIR / "test_clean.csv"

# Path file output final
TRAIN_FEATURES_PATH = FEATURES_DIR / "audio_train.npy"
TEST_FEATURES_PATH = FEATURES_DIR / "audio_test.npy"
TRAIN_LABELS_PATH = FEATURES_DIR / "audio_labels_train.npy"

# Membuat semua folder output yang dibutuhkan
FEATURES_DIR.mkdir(parents=True, exist_ok=True)
TEMP_TRAIN_DIR.mkdir(exist_ok=True)
TEMP_TEST_DIR.mkdir(exist_ok=True)

print("Semua path dan folder output sudah siap.")

# 2. FUNGSI EKSTRAKSI 
def extract_audio_features_from_wav(wav_path, n_mfcc=20):
    try:
        y, sr = librosa.load(wav_path)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        return np.mean(mfccs.T, axis=0)
    except Exception:
        return np.zeros(n_mfcc)

# TAHAP 1: EKSTRAK FITUR SECARA INDIVIDUAL (HEMAT MEMORI)
def process_and_save_individually(csv_path, wav_dir, output_dir):
    df = pd.read_csv(csv_path)
    for video_id in tqdm(df['id'], desc=f"Ekstrak & Simpan dari {wav_dir.name}"):
        wav_path = wav_dir / f"{video_id}.wav"
        feature_path = output_dir / f"{video_id}.npy"
        
        if feature_path.exists():
            continue
            
        if not wav_path.exists():
            features = np.zeros(20)
        else:
            features = extract_audio_features_from_wav(wav_path)
        
        np.save(feature_path, features)

print("\n--- TAHAP 1: Memulai ekstraksi fitur satu per satu ---")
process_and_save_individually(TRAIN_CSV_PATH, TRAIN_WAV_DIR, TEMP_TRAIN_DIR)
process_and_save_individually(TEST_CSV_PATH, TEST_WAV_DIR, TEMP_TEST_DIR)
print("Ekstraksi individual selesai.")

# TAHAP 2: GABUNGKAN SEMUA FILE FITUR
def combine_features(csv_path, temp_dir):
    df = pd.read_csv(csv_path)
    feature_list = []
    for video_id in tqdm(df['id'], desc=f"Menggabungkan fitur dari {temp_dir.name}"):
        feature = np.load(temp_dir / f"{video_id}.npy")
        feature_list.append(feature)
    return np.array(feature_list)

print("\n--- TAHAP 2: Menggabungkan semua hasil fitur ---")
# Gabungkan & simpan fitur train
train_features = combine_features(TRAIN_CSV_PATH, TEMP_TRAIN_DIR)
np.save(TRAIN_FEATURES_PATH, train_features)
print(f"Fitur training final disimpan di {TRAIN_FEATURES_PATH}")

# Gabungkan & simpan fitur test
test_features = combine_features(TEST_CSV_PATH, TEMP_TEST_DIR)
np.save(TEST_FEATURES_PATH, test_features)
print(f"Fitur testing final disimpan di {TEST_FEATURES_PATH}")

# TAHAP 3: BUAT FILE LABEL (Sama seperti sebelumnya)
print("\n--- TAHAP 3: Membuat file label ---")
train_df = pd.read_csv(TRAIN_CSV_PATH)
emotion_mapping = {
    'Proud': 0, 'Trust': 1, 'Joy': 2, 'Surprise': 3,
    'Neutral': 4, 'Sadness': 5, 'Fear': 6, 'Anger': 7
}
train_df['emotion_code'] = train_df['emotion'].map(emotion_mapping).fillna(train_df['emotion']).astype(int)
train_labels_np = train_df['emotion_code'].to_numpy()
np.save(TRAIN_LABELS_PATH, train_labels_np)
print(f"Label training berhasil disimpan di: {TRAIN_LABELS_PATH}")

print("\n\nSELESAI TOTAL! ✅ Semua file siap digunakan.")

Semua path dan folder output sudah siap.

--- TAHAP 1: Memulai ekstraksi fitur satu per satu ---


Ekstrak & Simpan dari test: 100%|██████████| 200/200 [00:24<00:00,  8.22it/s]


Ekstraksi individual selesai.

--- TAHAP 2: Menggabungkan semua hasil fitur ---


Menggabungkan fitur dari temp_train: 100%|██████████| 802/802 [00:08<00:00, 96.15it/s] 


Fitur training final disimpan di c:\Caelan\BDC2025\features\audio_train.npy


Menggabungkan fitur dari temp_test: 100%|██████████| 200/200 [00:02<00:00, 97.92it/s] 

Fitur testing final disimpan di c:\Caelan\BDC2025\features\audio_test.npy

--- TAHAP 3: Membuat file label ---
Label training berhasil disimpan di: c:\Caelan\BDC2025\features\audio_labels_train.npy


SELESAI TOTAL! ✅ Semua file siap digunakan.


### Output Ekstraksi Fitur

* **Visual**: `mp4 → fps → npy`
* **Text**: `mp4 → txt → npy`
* **Audio**: `mp4 → wav → npy`
* Semua hasil ekstraksi disimpan di folder **`features/`**


Video test extraction only for new datatest

Visual feature extraction

mp4 -> fps -> npy

In [7]:
import time
import numpy as np
import cv2
import pandas as pd
from pathlib import Path
import warnings

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tqdm import tqdm

warnings.filterwarnings("ignore")

# 1. SETUP MODEL & PATH
print("Memuat model ResNet50...")
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
visual_model = Model(inputs=base_model.input, outputs=base_model.output)
IMG_SIZE = 224
print("✅ Model Visual (ResNet50) siap.")

# --- Path Setup ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
VIDEO_DIR = DATA_DIR / "video"
PROCESSED_DATA_DIR = DATA_DIR / "processed"
FEATURES_DIR = BASE_DIR / "features"

# Path folder input video
TEST_VIDEO_DIR = VIDEO_DIR / "test"

# Path folder output sementara untuk menyimpan progres
TEMP_VISUAL_TEST_DIR = FEATURES_DIR / "temp_visual_test"

# Path file CSV (menggunakan datatest_revisi.csv)
TEST_CSV_PATH = PROCESSED_DATA_DIR / "datatest_revisi.csv"

# Path file output final (menggunakan nama file baru)
TEST_VISUAL_FEATURES_PATH = FEATURES_DIR / "visual_test_revisi.npy"

# Membuat semua folder output yang dibutuhkan
FEATURES_DIR.mkdir(exist_ok=True)
TEMP_VISUAL_TEST_DIR.mkdir(exist_ok=True)
print("Semua path dan folder output sudah siap.")

# 2. FUNGSI EKSTRAKSI (SAMA DENGAN KODE ASLI)
def extract_visual_features(video_path, num_frames=30):
    cap = cv2.VideoCapture(str(video_path))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames < 1: return np.zeros(2048)
    
    indices = np.linspace(0, total_frames - 1, num=num_frames, dtype=int)
    frames = []
    for i in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frames.append(frame)
    cap.release()
    
    if not frames: return np.zeros(2048)
    
    features = visual_model.predict(preprocess_input(np.array(frames)), batch_size=8, verbose=0)
    return np.mean(features, axis=0)

# 3. TAHAP 1: EKSTRAK & SIMPAN PROGRES PER FILE (HANYA UNTUK DATA TEST)
def process_and_save_individually(csv_path, video_dir, output_dir):
    df = pd.read_csv(csv_path)
    for video_id in tqdm(df['id'], desc=f"Ekstrak Visual dari {video_dir.name}"):
        video_path = video_dir / f"{video_id}.mp4"
        feature_path = output_dir / f"{video_id}.npy"
        
        if feature_path.exists():
            continue
            
        if not video_path.exists():
            features = np.zeros(2048)
        else:
            features = extract_visual_features(video_path)
        
        np.save(feature_path, features)

print("\n--- TAHAP 1: Memulai ekstraksi fitur visual satu per satu ---")
process_and_save_individually(TEST_CSV_PATH, TEST_VIDEO_DIR, TEMP_VISUAL_TEST_DIR)
print("✅ Ekstraksi individual selesai.")

# 4. TAHAP 2: GABUNGKAN SEMUA HASIL (HANYA UNTUK DATA TEST)
def combine_features(csv_path, temp_dir, output_path):
    df = pd.read_csv(csv_path)
    feature_list = []
    for video_id in tqdm(df['id'], desc=f"Menggabungkan fitur dari {temp_dir.name}"):
        feature = np.load(temp_dir / f"{video_id}.npy")
        feature_list.append(feature)
    
    final_features = np.array(feature_list)
    np.save(output_path, final_features)
    print(f"Fitur final disimpan di {output_path.name} dengan bentuk: {final_features.shape}")

print("\n--- TAHAP 2: Menggabungkan semua hasil fitur ---")
combine_features(TEST_CSV_PATH, TEMP_VISUAL_TEST_DIR, TEST_VISUAL_FEATURES_PATH)

print("\n\nSELESAI! ✅ Fitur visual siap digunakan.")

Memuat model ResNet50...
✅ Model Visual (ResNet50) siap.
Semua path dan folder output sudah siap.

--- TAHAP 1: Memulai ekstraksi fitur visual satu per satu ---


Ekstrak Visual dari test: 100%|██████████| 200/200 [14:11<00:00,  4.26s/it]


✅ Ekstraksi individual selesai.

--- TAHAP 2: Menggabungkan semua hasil fitur ---


Menggabungkan fitur dari temp_visual_test: 100%|██████████| 200/200 [00:03<00:00, 62.12it/s]

Fitur final disimpan di visual_test_revisi.npy dengan bentuk: (200, 2048)


SELESAI! ✅ Fitur visual siap digunakan.


Text feature extraction

mp4 -> txt

In [8]:
import pandas as pd
from pathlib import Path
import warnings
import json
import whisper
from tqdm import tqdm

warnings.filterwarnings("ignore")

# 1. SETUP MODEL & PATH
print("Memuat model Whisper...")
try:
    model = whisper.load_model("base")
    print("✅ Model Teks (Whisper) siap.")
except Exception as e:
    print(f"❌ Gagal memuat model Whisper: {e}")

# --- Path Setup ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
VIDEO_DIR = DATA_DIR / "video" 
PROCESSED_DATA_DIR = DATA_DIR / "processed"
FEATURES_DIR = BASE_DIR / "features"

# Path folder input video
TEST_VIDEO_DIR = VIDEO_DIR / "test"

# Path folder output sementara untuk menyimpan transkrip per file
TEMP_TEXT_TEST_DIR = FEATURES_DIR / "temp_text_test"

# Path file CSV
TEST_CSV_PATH = PROCESSED_DATA_DIR / "test_clean.csv"

# Path file output final 
TEST_TEXT_FEATURES_PATH = FEATURES_DIR / "text_test.csv"

# Membuat semua folder output yang dibutuhkan
FEATURES_DIR.mkdir(exist_ok=True)
TEMP_TEXT_TEST_DIR.mkdir(exist_ok=True)
print("Semua path dan folder output sudah siap.")

# 2. TAHAP 1: TRANSKRIPSI & SIMPAN PROGRES PER FILE
def transcribe_and_save_individually(csv_path, video_dir, output_dir):
    df = pd.read_csv(csv_path)
    for video_id in tqdm(df['id'], desc=f"Transkripsi dari {video_dir.name}"):
        video_path = video_dir / f"{video_id}.mp4"
        # Kita simpan hasilnya sebagai file teks biasa (.txt)
        transcript_path = output_dir / f"{video_id}.txt"
        
        # Cek apakah file ini sudah pernah diproses, jika iya, lewati
        if transcript_path.exists():
            continue
            
        transcript_text = ""
        if video_path.exists():
            try:
                # Lakukan transkripsi
                result = model.transcribe(str(video_path), fp16=False)
                transcript_text = result['text']
            except Exception as e:
                print(f"Gagal transkripsi {video_path.name}: {e}")
                transcript_text = "" # Beri teks kosong jika gagal
        
        # Simpan hasil transkrip ke file .txt
        with open(transcript_path, 'w', encoding='utf-8') as f:
            f.write(transcript_text)

print("\n--- TAHAP 1: Memulai transkripsi audio ke teks (TEST) ---")
transcribe_and_save_individually(TEST_CSV_PATH, TEST_VIDEO_DIR, TEMP_TEXT_TEST_DIR)
print("✅ Transkripsi individual selesai.")

# 3. TAHAP 2: GABUNGKAN SEMUA HASIL TRANSKRIP
def combine_transcripts(csv_path, temp_dir, output_path):
    df = pd.read_csv(csv_path)
    transcripts = []
    for video_id in tqdm(df['id'], desc=f"Menggabungkan transkrip dari {temp_dir.name}"):
        transcript_path = temp_dir / f"{video_id}.txt"
        try:
            with open(transcript_path, 'r', encoding='utf-8') as f:
                text = f.read()
        except FileNotFoundError:
            text = "" # Teks kosong jika file tidak ditemukan
        transcripts.append(text)
    
    # Buat DataFrame baru berisi id dan transkrip
    result_df = pd.DataFrame({
        'id': df['id'],
        'transcript': transcripts
    })
    
    # Simpan ke file CSV
    result_df.to_csv(output_path, index=False)
    print(f"Transkrip final disimpan di {output_path.name}")

print("\n--- TAHAP 2: Menggabungkan semua hasil transkrip (TEST) ---")
combine_transcripts(TEST_CSV_PATH, TEMP_TEXT_TEST_DIR, TEST_TEXT_FEATURES_PATH)

print("\n\nSELESAI! ✅ Transkrip teks (TEST) siap digunakan.")


Memuat model Whisper...
✅ Model Teks (Whisper) siap.
Semua path dan folder output sudah siap.

--- TAHAP 1: Memulai transkripsi audio ke teks (TEST) ---


Transkripsi dari test: 100%|██████████| 200/200 [47:21<00:00, 14.21s/it] 


✅ Transkripsi individual selesai.

--- TAHAP 2: Menggabungkan semua hasil transkrip (TEST) ---


Menggabungkan transkrip dari temp_text_test: 100%|██████████| 200/200 [00:03<00:00, 63.26it/s]


Transkrip final disimpan di text_test.csv


SELESAI! ✅ Transkrip teks (TEST) siap digunakan.


txt -> npy

In [9]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import torch
from tqdm import tqdm
from transformers import BertTokenizer, BertModel

warnings.filterwarnings("ignore")

# 1. SETUP MODEL INDOBERT & PATH
print("Mengecek ketersediaan GPU untuk PyTorch...")
# Cek apakah GPU bisa digunakan oleh PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan device: {device}")

print("\nMemuat model IndoBERT...")
try:
    # Muat tokenizer dan model IndoBERT
    tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
    model = BertModel.from_pretrained('indobenchmark/indobert-base-p1')
    # Pindahkan model ke GPU jika tersedia
    model.to(device)
    print("✅ Model IndoBERT siap.")
except Exception as e:
    print(f"❌ Gagal memuat model IndoBERT: {e}")

# --- Path Setup ---
BASE_DIR = Path.cwd().parent
FEATURES_DIR = BASE_DIR / "features"

# Path file input (hasil dari Whisper) -> hanya test
TEST_TEXT_CSV_PATH = FEATURES_DIR / "text_test.csv"

# Path file output final (.npy) -> hanya test
TEST_TEXT_FEATURES_PATH = FEATURES_DIR / "text_test.npy"

print("Semua path sudah siap.")

# 2. FUNGSI UNTUK MENDAPATKAN EMBEDDING
def get_text_embedding(text):
    """
    Mengubah satu kalimat teks menjadi satu vektor fitur (embedding).
    """
    if pd.isna(text) or text.strip() == "":
        # IndoBERT base menghasilkan 768 fitur
        return np.zeros(768) 

    # Tokenisasi teks
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=128, padding=True)
    # Pindahkan input tensor ke GPU
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Dapatkan output dari model tanpa menghitung gradien (lebih hemat memori)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Ambil embedding dari token [CLS] (representasi keseluruhan kalimat)
    embedding = outputs.last_hidden_state[0, 0, :].cpu().numpy()
    return embedding

# 3. PROSES DATASET TEST
def process_text_dataset(csv_path, output_path):
    df = pd.read_csv(csv_path)
    all_embeddings = []
    
    for text in tqdm(df['transcript'], desc=f"Membuat embedding dari {csv_path.name}"):
        embedding = get_text_embedding(text)
        all_embeddings.append(embedding)
    
    # Simpan hasil akhir ke file .npy
    final_embeddings = np.array(all_embeddings)
    np.save(output_path, final_embeddings)
    print(f"Fitur teks final disimpan di {output_path.name} dengan bentuk: {final_embeddings.shape}")

print("\n--- Memulai proses embedding untuk data tes ---")
process_text_dataset(TEST_TEXT_CSV_PATH, TEST_TEXT_FEATURES_PATH)

print("\n\nSELESAI! ✅ Fitur teks dari data test sudah siap.")


Mengecek ketersediaan GPU untuk PyTorch...
Menggunakan device: cpu

Memuat model IndoBERT...
✅ Model IndoBERT siap.
Semua path sudah siap.

--- Memulai proses embedding untuk data tes ---


Membuat embedding dari text_test.csv: 100%|██████████| 200/200 [00:33<00:00,  5.89it/s]

Fitur teks final disimpan di text_test.npy dengan bentuk: (200, 768)


SELESAI! ✅ Fitur teks dari data test sudah siap.


Audio feture extraction

mp4 -> wav

In [11]:
import os
import subprocess
from pathlib import Path
from tqdm import tqdm

# --- Konfigurasi Path ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
VIDEO_DIR = DATA_DIR / "video"
WAV_DIR = DATA_DIR / "wav"  # Folder output untuk file .wav

# Hanya proses folder test
video_subdirs = [VIDEO_DIR / "test"]

print("Memulai proses konversi MP4 ke WAV...")

# Loop hanya untuk folder test
for video_subdir in video_subdirs:
    # Tentukan folder output untuk file .wav
    wav_output_dir = WAV_DIR / video_subdir.name
    # Buat folder output jika belum ada
    wav_output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\nMemproses folder: {video_subdir}")
    
    # Dapatkan daftar semua file .mp4 di dalam subdirektori
    video_files = list(video_subdir.glob("*.mp4"))
    
    for video_path in tqdm(video_files, desc=f"Mengonversi {video_subdir.name}"):
        video_id = video_path.stem  # Mengambil nama file tanpa ekstensi (misal: "1")
        wav_path = wav_output_dir / f"{video_id}.wav"
        
        # Lewati file yang sudah pernah dikonversi
        if wav_path.exists():
            continue
            
        # Perintah FFMPEG untuk konversi
        command = [
            "ffmpeg",
            "-i", str(video_path),
            "-vn",
            "-acodec", "pcm_s16le",
            "-ar", "22050",
            "-ac", "1",
            str(wav_path),
            "-hide_banner",  # Sembunyikan info banner ffmpeg
            "-loglevel", "error"  # Hanya tampilkan error jika ada
        ]
        
        try:
            # Menjalankan perintah ffmpeg
            subprocess.run(command, check=True)
        except subprocess.CalledProcessError as e:
            print(f"Gagal mengonversi {video_path.name}: {e}")

print("\n\nSemua video (TEST) berhasil dikonversi ke format WAV! 🚀")


Memulai proses konversi MP4 ke WAV...

Memproses folder: c:\Caelan\BDC2025\data\video\test


Mengonversi test: 100%|██████████| 200/200 [01:00<00:00,  3.33it/s]



Semua video (TEST) berhasil dikonversi ke format WAV! 🚀


wav -> npy

In [12]:
import pandas as pd
import numpy as np
import librosa
from pathlib import Path
from tqdm import tqdm
import warnings

# Abaikan warning dari librosa yang tidak relevan
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# 1. SETUP PATH
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
WAV_DIR = DATA_DIR / "wav"
PROCESSED_DATA_DIR = DATA_DIR / "processed"
FEATURES_DIR = BASE_DIR / "features"

# Path folder input WAV
TEST_WAV_DIR = WAV_DIR / "test"

# Path folder output sementara untuk .npy per file
TEMP_TEST_DIR = FEATURES_DIR / "temp_test"

# Path file CSV
TEST_CSV_PATH = PROCESSED_DATA_DIR / "test_clean.csv"

# Path file output final
TEST_FEATURES_PATH = FEATURES_DIR / "audio_test.npy"

# Membuat semua folder output yang dibutuhkan
FEATURES_DIR.mkdir(parents=True, exist_ok=True)
TEMP_TEST_DIR.mkdir(exist_ok=True)

print("Semua path dan folder output sudah siap.")

# 2. FUNGSI EKSTRAKSI 
def extract_audio_features_from_wav(wav_path, n_mfcc=20):
    try:
        y, sr = librosa.load(wav_path)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        return np.mean(mfccs.T, axis=0)
    except Exception:
        return np.zeros(n_mfcc)

# TAHAP 1: EKSTRAK FITUR SECARA INDIVIDUAL (HEMAT MEMORI)
def process_and_save_individually(csv_path, wav_dir, output_dir):
    df = pd.read_csv(csv_path)
    for video_id in tqdm(df['id'], desc=f"Ekstrak & Simpan dari {wav_dir.name}"):
        wav_path = wav_dir / f"{video_id}.wav"
        feature_path = output_dir / f"{video_id}.npy"
        
        if feature_path.exists():
            continue
            
        if not wav_path.exists():
            features = np.zeros(20)
        else:
            features = extract_audio_features_from_wav(wav_path)
        
        np.save(feature_path, features)

print("\n--- TAHAP 1: Memulai ekstraksi fitur TEST satu per satu ---")
process_and_save_individually(TEST_CSV_PATH, TEST_WAV_DIR, TEMP_TEST_DIR)
print("Ekstraksi individual selesai.")

# TAHAP 2: GABUNGKAN SEMUA FILE FITUR TEST
def combine_features(csv_path, temp_dir):
    df = pd.read_csv(csv_path)
    feature_list = []
    for video_id in tqdm(df['id'], desc=f"Menggabungkan fitur dari {temp_dir.name}"):
        feature = np.load(temp_dir / f"{video_id}.npy")
        feature_list.append(feature)
    return np.array(feature_list)

print("\n--- TAHAP 2: Menggabungkan semua hasil fitur TEST ---")
test_features = combine_features(TEST_CSV_PATH, TEMP_TEST_DIR)
np.save(TEST_FEATURES_PATH, test_features)
print(f"Fitur testing final disimpan di {TEST_FEATURES_PATH}")

print("\n\nSELESAI TOTAL! ✅ Semua file TEST siap digunakan.")


Semua path dan folder output sudah siap.

--- TAHAP 1: Memulai ekstraksi fitur TEST satu per satu ---


Ekstrak & Simpan dari test: 100%|██████████| 200/200 [00:33<00:00,  6.03it/s]


Ekstraksi individual selesai.

--- TAHAP 2: Menggabungkan semua hasil fitur TEST ---


Menggabungkan fitur dari temp_test: 100%|██████████| 200/200 [00:03<00:00, 59.65it/s]

Fitur testing final disimpan di c:\Caelan\BDC2025\features\audio_test.npy


SELESAI TOTAL! ✅ Semua file TEST siap digunakan.
